In [1]:
import os
import sys
# path to settings
sys.path.insert(0, os.path.split(os.getcwd())[0])
# path to src/models
sys.path.insert(1, os.path.join(os.path.split(os.getcwd())[0], 'src/models/'))

from settings import MODEL_DIR, FIGURES_DIR, REAL_DATA_FILTER_SPEC, FILTER_SPECTROGRAM_DIR
from keras.preprocessing import image
from sklearn.metrics import confusion_matrix
from model import get_model
import re
import matplotlib.pyplot as plt
import keras
import itertools
import numpy as np

Using TensorFlow backend.


In [2]:
# took from scikit-learn.org example
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [3]:
import re

def natural_sort(l):
    '''
    needed in order to sort numbers and strings
    '''
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(l, key = alphanum_key)

In [93]:
file = open('rezultati', 'a')
def save_list_to_file(file, predicted_list, filename):
    file.write('\n#' + filename + '\n')
    file.write(str(predicted_list))

In [94]:
# dimensions of our images
img_height, img_width = 480, 20
input_shape = (img_height, img_width, 3)
num_classes = 8

class_labels = [[
        'm0', 'm1', 'm2', 'm3', 'm4', 'm5', 'blank'
    ], [
        'v0', 'v1', 'v2', 'v3', 'v4', 'v5', 'blank'
    ]
]

# model must be the same as trained
model = get_model(input_shape, num_classes)

# load the model we saved
model.load_weights(MODEL_DIR + 'm/'+ 'model_20_other.h5')
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adadelta(),
    metrics=['accuracy']
)

# use test set
test_datagen = image.ImageDataGenerator(rescale=1. / 255)

batch_size = 50
test_batches = test_datagen.flow_from_directory(
    REAL_DATA_FILTER_SPEC,
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical'
)


true_classes = []
predicted_classes = []

folders = os.listdir(REAL_DATA_FILTER_SPEC + 'm/')
folders.sort()

for i, folder in enumerate(folders[-3:]):

    class_folder = REAL_DATA_FILTER_SPEC + 'm/' + folder
    class_image_data = []

    files_sorted = os.listdir(class_folder)
    files_sorted = natural_sort(files_sorted)
    for data in files_sorted:

        image_path = class_folder + '/' + data

        img = image.load_img(image_path, target_size=(img_height, img_width))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        class_image_data.append(x)

    # stack arrays vertically
    array_row = np.vstack(class_image_data)

    # predicted class
    predicted_class = model.predict_classes(array_row, batch_size=20)
    save_list_to_file(file, predicted_class, folder)
    print(folder)
    print(predicted_class)
file.close()

Found 4736 images belonging to 2 classes.
mala23brzo
[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 2 2 2 2 2 3 3 2 2 2 3 3
 2 2 2 3 3 3 2 2 2 2 3 3 2 2 2 3 3 3 2 3 3 3 3 3 2 2 2 3 3 3 2 2 3 3 3 2 2
 3 3 3 2 2 2 2 3 3 3 2 2 2 3 3 2 2 2 2 3 3 3 2 2 2 2 3 3 2 2 2 2 3 2 2 2 3
 3 3 2 2 2 2 2 3 3 3 2 2 2 3 3 3 3 3 3 2 2 2 2 2 3 3 2 2 2 2 3 3 3 2 2 2 3
 3 3 2 2 2 3 3 3 2 2 2 3 3 3 3 2 2 2 3 3 3 2 2 2 3 3 3 2 2 2 3 3 2 2 2 3 3
 3 3 3 3 3 3 3 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7]
mala5do0brzo
[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 6 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 4 4 4 2 1 0 0 0
 0 0 0 0 0 0 0 0 0 6 0 1 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
mala5do0do5
[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 

In [135]:
file = open('rezultati', 'r')

def extract_list(file):
    extracted_list = []
    curr_list = ''
    
    for line in file.readlines():

        if '#' in line:
            title = line.lstrip(' #')
            continue
            
        if '[' in line:
            curr_list = str(line.lstrip('[').rstrip('\n]'))
            continue
            #print(curr_list)
            
        
        if ']' in line and curr_list != '':
            #print(curr_list)
            curr_list = curr_list.split(' ')
            #print(curr_list)
            curr_list = [int(i) for i in curr_list]
            extracted_list.append((title, curr_list))
            curr_list = ''
            continue
            
        if curr_list != '':
            curr_list += line.rstrip('\n')
            
    
    return extracted_list
            
results = extract_list(file)
#print(results)

In [134]:
from sty import fg, bg, ef, rs, Rule

silence = lambda txt: ('%s%s%s%s%s') % (bg.white, fg.black, txt, fg.rs, bg.rs)
other = lambda txt: ('%s%s%s%s%s') % (bg.black, fg.white, txt, fg.rs, bg.rs)
tone_0 = lambda txt: ('%s%s%s%s%s') % (bg.blue, fg.black, txt, fg.rs, bg.rs)
tone_1 = lambda txt: ('%s%s%s%s%s') % (bg.green, fg.black, txt, fg.rs, bg.rs)
tone_2 = lambda txt: ('%s%s%s%s%s') % (bg.cyan, fg.black, txt, fg.rs, bg.rs)
tone_3 = lambda txt: ('%s%s%s%s%s') % (bg.red, fg.black, txt, fg.rs, bg.rs)
tone_4 = lambda txt: ('%s%s%s%s%s') % (bg.yellow, fg.black, txt, fg.rs, bg.rs)
tone_5 = lambda txt: ('%s%s%s%s%s') % (bg.magenta, fg.black, txt, fg.rs, bg.rs)

COLORS = [
    tone_0,
    tone_1,
    tone_2,
    tone_3,
    tone_4,
    tone_5,
    other,
    silence 
]

COLORS_TXT = [
    ('tone 0', 'blue'), ('tone 1', 'green'), ('tone 2', 'cyan'), 
    ('tone 3', 'red'), ('tone 4', 'yellow'), ('tone 5', 'magenta'), 
    ('other', 'black'), ('silence', 'white') 
]

# color legend
for i, color in enumerate(COLORS):
    print(COLORS_TXT[i][0] + '=>' + color(COLORS_TXT[i][1]))
print('\n---------------\n\n')         
def format_output(predictions_list):
    formated_string = ''
    
    for prediction in predictions_list:    
        formated_string += COLORS[prediction](str(prediction))
    
    return formated_string
        
for result in results:  
    print(result[0])
    print(format_output(result[1]))
    print('\n')

tone 0=>blue
tone 1=>green
tone 2=>cyan
tone 3=>red
tone 4=>yellow
tone 5=>magenta
other=>black
silence=>white

---------------


mala23brzo

77777777777777777777777777777777777777777777777777777777777722222222222222222222222222222222222223322222332223322233322223322233323333322233322333223332222333222332222333222233222232223332222233322233333322222332222333222333222333222333322233322233322233222333333333777777777777777777777777777777


mala5do0brzo

777777777777777777777777777777777777777765555555555555555555555555444210000000000006017777777777777777777777777


mala5do0do5

7777777777777777777777777777777777777777777777777555555555555555555555555555555555555555555555444444444444444444444444444444444444444444444444443333333333333333333333333333333333333333333333333322222222222222222222222222222222222222222222222222211111111111111111111111111111111111111111111111111111111010110000000000000000000000000000000000000000000000000001777777777777777777777777777777777777777777777777777777777777

In [23]:
predicted_class


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 4, 4, 4,
       4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 3, 6, 6, 3, 6, 3, 3, 3, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,